<a href="https://colab.research.google.com/github/MahmoudHussienMohamed/Flask-ngrok-APIs/blob/VirtualFittingRoomAPI/ACGPN_APIv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/ACGPN-API-Files/ACGPN

/content/drive/.shortcut-targets-by-id/10Nut25hcbPBd9RKDpSY4ntS-3LgnABBW/ACGPN-API-Files/ACGPN


In [ ]:
!pip install ninja
!pip install flask
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 32.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gdown
import numpy as np
from PIL import Image
import IPython
import gdown
import os
import sys
from predict_pose import generate_pose_keypoints

In [ ]:
%cd U-2-Net
import u2net_load
import u2net_run
u2net = u2net_load.model(model_name = 'u2netp')
%cd ..

/content/drive/.shortcut-targets-by-id/10Nut25hcbPBd9RKDpSY4ntS-3LgnABBW/ACGPN-API-Files/ACGPN/U-2-Net
...load U2NEP---4.7 MB
/content/drive/.shortcut-targets-by-id/10Nut25hcbPBd9RKDpSY4ntS-3LgnABBW/ACGPN-API-Files/ACGPN


In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | \
sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && \
echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | \
sudo tee /etc/apt/sources.list.d/ngrok.list && \
sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [824 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Hit

In [ ]:
# !curl -L https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -o ngrok.zip
# !unzip ngrok.zip
# !chmod +x ngrok
!./ngrok authtoken <YourAuthToken>

/bin/bash: ./ngrok: Permission denied


In [ ]:
from flask import Flask, send_file
from flask_ngrok import run_with_ngrok
from flask import request, jsonify
import json
import base64
from io import BytesIO

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/', methods=['GET'])
def index():
    return jsonify({'VirtualFittingRoom-API': "Version 1.0"})



@app.route('/apply', methods=['POST'])
def apply():
  if 'person_image' not in request.files :
      return  jsonify({'error': "error-key_notfound-person_image" })
  if 'clothing_image' not in request.files :
      return  jsonify({'error': "error-key_notfound-clothing_image" })
  
  clothing_img = request.files['clothing_image']
  clothing_img = Image.open(clothing_img.stream)
  clothing_img = clothing_img.resize((500, 667), Image.BICUBIC).convert('RGB')
  clothing_img.save(os.path.join('Data_preprocessing/test_color', '000001_1.png'))
  u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge')

  person_img = request.files['person_image']
  person_img_name = '000001_0.png'
  person_img = Image.open(person_img.stream)
  person_img.save('inputs/img/000001_0.png')
  person_img_path = os.path.join('inputs/img', sorted(os.listdir('inputs/img'))[0])
  person_img = person_img.resize((500,667), Image.BICUBIC)
  person_img.save(os.path.join('Data_preprocessing/test_img', person_img_name))
  
  os.system("python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py --dataset 'lip' --model-restore 'lip_final.pth' --input-dir 'Data_preprocessing/test_img' --output-dir 'Data_preprocessing/test_label'")
  pose_path = os.path.join('Data_preprocessing/test_pose', person_img_name.replace('.png', '_keypoints.json'))
  generate_pose_keypoints(person_img_path, pose_path)
  os.system("rm -rf Data_preprocessing/test_pairs.txt")
  with open('Data_preprocessing/test_pairs.txt','w') as f:
      f.write('000001_0.png 000001_1.png')
  os.system("python test.py")
  img = Image.open('results/test/try-on/000001_0.png') 
  buff = BytesIO()
  img.save(buff, format="PNG")
  img_str = base64.b64encode(buff.getvalue()).decode('ascii')
  return jsonify({'result' : str(img_str)})
  
@app.route('/clean', methods=['POST'])
def clean():
  os.system("rm -rf Data_preprocessing/test_color/*")
  os.system("rm -rf Data_preprocessing/test_colormask/*")
  os.system("rm -rf Data_preprocessing/test_edge/*")
  os.system("rm -rf Data_preprocessing/test_img/*")
  os.system("rm -rf Data_preprocessing/test_label/*")
  os.system("rm -rf Data_preprocessing/test_mask/*")
  os.system("rm -rf Data_preprocessing/test_pose/*")
  os.system("rm -rf inputs/cloth/*")
  os.system("rm -rf inputs/img/*")
  os.system("rm -rf results/*")
  return jsonify({"sucess" : "Cleaned"})

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e88a-35-198-244-100.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Generating mask for: 000001_1.png
Saving output at Data_preprocessing/test_edge/000001_1.png
File saved at Data_preprocessing/test_pose/000001_0_keypoints.json


127.0.0.1 - - [27/Jul/2022 23:46:16] "POST /apply HTTP/1.1" 200 -


Generating mask for: 000001_1.png
Saving output at Data_preprocessing/test_edge/000001_1.png
File saved at Data_preprocessing/test_pose/000001_0_keypoints.json


...

127.0.0.1 - - [28/Jul/2022 00:05:49] "POST /apply HTTP/1.1" 200 -


Generating mask for: 000001_1.png
Saving output at Data_preprocessing/test_edge/000001_1.png
File saved at Data_preprocessing/test_pose/000001_0_keypoints.json


127.0.0.1 - - [28/Jul/2022 00:06:09] "POST /apply HTTP/1.1" 200 -


Generating mask for: 000001_1.png
Saving output at Data_preprocessing/test_edge/000001_1.png
File saved at Data_preprocessing/test_pose/000001_0_keypoints.json


127.0.0.1 - - [28/Jul/2022 00:06:31] "POST /apply HTTP/1.1" 200 -
